# Feature Engineering and Preprocessing

## Importing Libraries and Loading Data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.append('../utils')

from MDLP import MDLP_Discretizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.linear_model import LogisticRegression

%matplotlib inline

In [2]:
df = pd.read_csv('../data/processed/sales_cleaned.csv')
df.head()

,X1_cor_sales_in_vol,X2_cor_sales_in_val,X3_CA_mag,X4_value,X6_VenteConv,X5_ENSEIGNE,X7_Feature,Y_Display
0,2.0,20.20,47400,36,72.0,CORA,No_Feat,No_Displ
1,2.0,11.90,62000,24,48.0,LECLERC,No_Feat,No_Displ
2,8.0,29.52,60661,60,480.0,AUCHAN,No_Feat,No_Displ
3,2.0,16.20,59677,19,38.0,CARREFOUR,No_Feat,No_Displ
4,5.0,62.10,142602,50,250.0,CORA,No_Feat,No_Displ


## Preprocessing

Before applying any machine learning model on our dataset, we'll want to compare between two versions of the data, one using the original numerical features, and one using a categorical version of these features, by binning them based on the information gain criterion. This process is called MDLPC.  
Let's create our preprocessors to use in pipelines with ml models. We'll mainly have two:
- OHE for categorical features followed by a standard scaler for the numerical features.
- OHE for categorical features followed by a MDLP discretizer for the numerical features.